# Decision Trees  
Decision Trees are **non-parametric supervised learning** methods that can be used for modelling **classification** as well as **regression** problems. 

> partitions the feature space into a set of rectangles (or cuboid) and then fit a simple model (like a constant) in each one.

1. Binary Decision tree
  - splits into two branches at each node.  

Important Issues:
1. Which attribute to choose for splitting?
2. What should be the splitting criterion?
3. What tree size will give the optimal solution?




**ID3 (Iterative Dichotomizer 3) Algorithm for Decision Trees**
![ID3 tree.png](https://drive.google.com/uc?id=1ugdBXPEiXgBkKGXYnmkOiFFfCd9T5AmQ)  

Recursion on a subset may stop by any of the **Stopping criteria**.  


- Impurity Measures
  - Proportion of data in node i,  
  $p_{i,k} = \frac{1}{N _{i}} \sum \limits _{x ^{(i)} \in R _{i}} 1(y ^{(i)} = k) $ &emsp; &emsp; _$N _{i}$ is number of samples in region $R _{i}$_
  - Misclassification Error
  - Gini Index
  - Entropy, (means **randomness**)  
  $H_{i} = - \sum \limits _{k=1} ^{n} p _{i,k} log _{2} p _{i,k}$ 
  - Information gain
  ![Information gain formula](https://drive.google.com/uc?id=13_8cr777OUcJjdEEVz4w70728WhnViqy)

In ID3, information gain can be calculated (instead of entropy) for each remaining attribute. The attribute with the **largest** information gain is used to split the dataset on this iteration.  



### CART  
- CART can model both **regression** and **classification** problems.  
- CART learns **decision trees** for **classification** very similar to **ID3**.



### Overfitting in Decision Trees  
A very large tree might overfit the data.

Methods to avoid overfitting:
- Pre-pruning
- Cost-complexity pruning  
- Post-pruning

### Advantages of Decision Trees:
- Versatile, can perform classification, regression and multi-output tasks.
- Simple to understand and interpret. Trees can be visualised.
- Needs little data preprocessing.
- Cost of using tree is logarithmic in the number of training data samples.  

### Disadvantages of Decision Trees:
- Can create overly complex trees that do not generalize to unseen data. Pruning is needed to address this issue.
- Decision trees suffer from the problem of high variance, which can be mitigated by ensembles.
- Trees learn peicewise linear approximation and hence are not good at extrapolation.


___

## Implementation of Decision tree for classification with ID3 algorithm:

#### Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

Here `eps` is the smallest representable number. At times we get `log(0)` or `0` in the denominator, to avoid that we are going to use this.

#### Demo: Classification
In this case we will use a synthetic data set for classification task.

In [ ]:
#@title [Load Dataset]

# mount Google drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


# read file
path = "/content/drive/MyDrive/Colab Notebooks/Datasets/tennis.csv"
df = pd.read_csv(path)

Mounted at /content/drive


In [ ]:
df.shape

(14, 5)

In [ ]:
df.values

array([['sunny', 'hot', 'high', False, 'no'],
       ['sunny', 'hot', 'high', True, 'no'],
       ['overcast', 'hot', 'high', False, 'yes'],
       ['rainy', 'mild', 'high', False, 'yes'],
       ['rainy', 'cool', 'normal', False, 'yes'],
       ['rainy', 'cool', 'normal', True, 'no'],
       ['overcast', 'cool', 'normal', True, 'yes'],
       ['sunny', 'mild', 'high', False, 'no'],
       ['sunny', 'cool', 'normal', False, 'yes'],
       ['rainy', 'mild', 'normal', False, 'yes'],
       ['sunny', 'mild', 'normal', True, 'yes'],
       ['overcast', 'mild', 'high', True, 'yes'],
       ['overcast', 'hot', 'normal', False, 'yes'],
       ['rainy', 'mild', 'high', True, 'no']], dtype=object)

In [ ]:
df.keys()

Index(['outlook', 'temp', 'humidity', 'windy', 'play'], dtype='object')

In [ ]:
target = df.keys()[-1]
target

'play'

Lets check total number of labels.

In [ ]:
df[target].unique()

array(['no', 'yes'], dtype=object)

In [ ]:
# Number of 'no' and 'yes'
print(df[target].value_counts()[df[target].unique()[0]])
print(df[target].value_counts()[df[target].unique()[1]])

5
9


### Calculating Entropy of the whole dataset  
Since the formula for **information gain** requires the entropy of the whole dataset, we compute that now:

In [ ]:
def findEntropyWhole(df):
  #last column in the dataframe is the target variable
  target = df.keys()[-1]

  #initailization
  overall_entropy = 0

  #possible values of target
  values_in_target = df[target].unique()

  for value in values_in_target:
    p = df[target].value_counts()[value]/len(df[target])
    overall_entropy += -p*np.log2(p)
  return overall_entropy

findEntropyWhole(df)

0.9402859586706309

###Calculating entropy of an attribute

In [ ]:
def findEntropyAttribute(df, attribute):
  #last column in the dataframe is the target variable
  target = df.keys()[-1]

  #initailization
  overall_entropy = 0

  #possible values of target
  values_in_target = df[target].unique()

  #this gives different features in that attribute like hot, cold in temperature
  values_in_attribute = df[attribute].unique()

  #initialize attribute entropy
  entropy_attribute = 0

  for value_in_attribute in values_in_attribute:
    overall_entropy = 0
    for value_in_target in values_in_target:
      num = len(df[attribute][df[attribute] == value_in_attribute][df[target] == value_in_target])
      den = len(df[attribute][df[attribute] == value_in_attribute])
      p = num/(den+eps)
      overall_entropy += -p*np.log2(p+eps)
    p2 = den/len(df)
    entropy_attribute += -p2*overall_entropy
  return abs(entropy_attribute)

Let's compute entropy of different attributes now:

In [ ]:
for attribute in df.keys()[:-1]:
  print('Entropy of attribute {}: {}'.format(attribute, findEntropyAttribute(df, attribute)))

Entropy of attribute outlook: 0.6935361388961914
Entropy of attribute temp: 0.9110633930116756
Entropy of attribute humidity: 0.7884504573082889
Entropy of attribute windy: 0.892158928262361


Finding the best attribute

In [ ]:
def find_best_attribute_to_divide(df):
  #Information gain
  IG = []

  # all column names
  all_attributes_names = df.keys()[:-1]

  for attribute in all_attributes_names:
    # compute information gain for every attribute
    IG.append(findEntropyWhole(df)- findEntropyAttribute(df, attribute))

    # get index of attribute with best IG
    max_IG_attribute_index = np.argmax(IG)

    # best attribute name
    best_attribute = all_attributes_names[max_IG_attribute_index]
  return best_attribute

find_best_attribute_to_divide(df)

'outlook'

#### Building the Decision tree

In [ ]:
def build_tree(df, tree=None):
  # last column in dataframe
  target = df.keys()[-1]

  # Here we build our Decision tree

  # get attribute with max IG
  node = find_best_attribute_to_divide(df)

  # get distinct values of that attribute
  attributeValues = np.unique(df[node])

  # create an empty dictionary to create tree
  if tree is None:
    tree = {}
    tree[node] = {}

  # we make loop to construct a tree by calling this function recursively
  # in this we check if the subset is pure and stops if it is pure
  for value in attributeValues:
    subtable = df[df[node] == value].reset_index(drop=True)
    clValue, counts = np.unique(subtable['play'], return_counts=True)
    if len(counts) == 1: # Checking purity of subset
      tree[node][value] = clValue[0]
    else:
      tree[node][value] = build_tree(subtable) # calling the function recursively
  return tree

build_tree(df)

{'outlook': {'overcast': 'yes',
  'rainy': {'windy': {False: 'yes', True: 'no'}},
  'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}

ID3 in its pure form performs no backtracking in its search. Once it selects an attribute to test at a particular level in the tree, it never backtracks to reconsider this choice. Therefore, it is susceptible to the usual risk of hill-climbing search without backtracking: converging to local optimal solutions that may not be globally optimal.

### Interview questions
1. Entropy vs Gini Index vs IG
2. When to use what?
3. Decision tree in Regression

In [4]:
x = [range(5)]
x

[range(0, 5)]

In [ ]:
x